In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameter Estimation
In this notebook I attempt to approach the designation of parameters anew. The main goal is to make clear the source of all parameters, be it from scientific article, or from a computation. The purpose is to make the research more accessible/understandable, as well as ensuring correctness of parameters following a parameter change.

In [18]:
SHARED_PARAMS = {
    "bbb": [
        ("arteries", "pvs_arteries"),
        ("capillaries", "pvs_capillaries"),
        ("veins", "pvs_veins")
    ],
    "aef": [
        ("pvs_arteries", "ecs"),
        ("pvs_capillaries", "ecs"),
        ("pvs_veins", "ecs")
    ],
    "pvs": ["pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "csf": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "blood": ["arteries", "capillaries", "veins"],
    "all": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins", "arteries", "capillaries", "veins"]
}

In [19]:
def retrieve_aliases(param):
    split = param.split("-")
    return [word for word in split[1:] if word in SHARED_PARAMS]


def unpack_shared_parameters(parameters):
    newparameters = {}
    for param, value in parameters.items():
        aliases = retrieve_aliases(param)
        for alias in aliases:
            if type(SHARED_PARAMS[alias][0]) == str:
                newnames = [param.replace(alias, key) for key in SHARED_PARAMS[alias]]
            else:
                newnames = [param.replace(alias, "-".join(key)) for key in SHARED_PARAMS[alias]]
            
            for name in newnames:
                newparameters[name] = value
        
        if len(aliases) == 0:
            newparameters[param] = value
        
    return newparameters

In [20]:
from pint import UnitRegistry
from pint import Quantity
from multirat.parameters import unpack_shared_parameters

ureg = UnitRegistry()

mmHg = ureg.mmHg
Pa = ureg.Pa
mm = ureg.mm
s = ureg.s

In [56]:
1 / mm

1 <Unit('1 / millimeter')>

In [55]:
base_parameters = {
    # Brain Volume
    "brain_volume": 2450. * mm**3,
    # Pressure boundary parameters
    "pressure_boundary-pvs_arteries": 4.74 * mmHg,
    "pressure_boundary-ecs": 3.74 * mmHg,
    "pressure_boundary-pvs_veins": 3.36 * mmHg,
    "pressure_boundary-arteries": 120. * mmHg,
    "pressure_boundary-veins": 7.0 * mmHg,
    "diffusion_free_inulin-all": 2.98 * mm**2 / s,
    "tortuosity-all": 1.7,
    "osmotic_pressure-blood": 20. * mmHg,
    "permeability_inulin-bbb": 0.0 * mm / s,  # Does not cross the BBB
    "porosity-ecs": 0.14,
    "vasculature_volume_fraction": 0.0329,
    "vasculature_fraction-arteries": 0.2,  #  (Lee et al. 2001)
    "vasculature_fraction-capillaries": 0.1, # (Lee et al. 2001)
    "vasculature_fraction-veins": 0.7,  # (Lee et al. 2001)
    "pvs_volume_fraction":  0.003,  # (Ballerini, et al. 2020)
    "viscosity-blood": 2.67e-3 * Pa*s,  #  (Guo et al. 2019)
    "viscosity-csf": 7.0e-4 * Pa*s,  #  (Bloomfield, Johnston, Bilston 1998)
    "permeability-ecs": 2.0e-11 * mm**2,
    "resistance-ecs": 4.56 * Pa * s / mm**2,
    # Compute more permeabilities
    "hydraulic_conductivity-ecs-arteries": 9.1e-10 * mm / (Pa * s),
    "hydraulic_conductivity-ecs-capillaries": 1.0e-10 * mm / (Pa * s),
    "hydraulic_conductivity-ecs-veins": 2.0e-11 * mm / (Pa * s),
    "surface_volume_ratio-ecs-arteries": 3. * 1 / mm,   # (Assumed 1/3 of capillary)
    "surface_volume_ratio-ecs-capillaries": 9. * 1 / mm,  # (Smith and Humphrey 2007)
    "surface_volume_ratio-ecs-veins": 3. * 1 / mm,  # (Assumed 1/3 of capillary)
}

base_parameters

{'brain_volume': 2450.0 <Unit('millimeter ** 3')>,
 'pressure_boundary-pvs_arteries': 4.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-ecs': 3.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-pvs_veins': 3.36 <Unit('millimeter_Hg')>,
 'pressure_boundary-arteries': 120.0 <Unit('millimeter_Hg')>,
 'pressure_boundary-veins': 7.0 <Unit('millimeter_Hg')>,
 'diffusion_free_inulin-all': 2.98 <Unit('millimeter ** 2 / second')>,
 'tortuosity-all': 1.7,
 'osmotic_pressure-blood': 20.0 <Unit('millimeter_Hg')>,
 'permeability_inulin-bbb': 0.0 <Unit('millimeter / second')>,
 'porosity-ecs': 0.14,
 'vasculature_volume_fraction': 0.0329,
 'vasculature_fraction-arteries': 0.2,
 'vasculature_fraction-capillaries': 0.1,
 'vasculature_fraction-veins': 0.7,
 'pvs_volume_fraction': 0.003,
 'viscosity-blood': 0.00267 <Unit('pascal * second')>,
 'viscosity-csf': 0.0007 <Unit('pascal * second')>,
 'permeability-ecs': 2e-11 <Unit('millimeter ** 2')>,
 'resistance-ecs': 4.56 <Unit('pascal * second / millimeter 

In [ ]:
# TODO: Derive these parameters properly!
# FIX: Check units!
cheating_parameters = {
    "transfer-ecs-pvs_arteries": 2.1932017763179826e-07,
    "transfer-ecs-pvs_capillaries": 9.976258977745193e-10,
    "transfer-ecs-pvs_veins": 1.9533203320332029e-07,
    "transfer-pvs_arteries-arteries": 5.89e-09,
    "transfer-pvs_arteries-pvs_capillaries": 2.5002500250025003e-08,
    "transfer-pvs_capillaries-capillaries": 2.98e-09,
    "transfer-pvs_capillaries-pvs_veins": 2.5002500250025003e-08,
    "transfer-pvs_veins-veins": 1.26e-10,
    "transfer-arteries-capillaries": 1.05010501050105e-07,
    "transfer-capillaries-veins": 5.250525052505252e-07
}

In [34]:
def effective_diffusion(D_free, tortuosity):
    return D_free / tortuosity**2


def get_effective_diffusion(p):
    return {
        f"diffusion_eff_inulin-{x}":
            effective_diffusion(
                p[f"diffusion_free_inulin-{x}"],
                p[f"tortuosity-{x}"]
            )
        for x in SHARED_PARAMS["all"] if x in p
    }


def subnetwork_porosity(network_volume_fraction, subnetwork_fraction):
    return network_volume_fraction * subnetwork_fraction


def get_vasculature_porosity(p):
    return {
        f"porosity-{x}":
            subnetwork_porosity(
                p['vasculature_volume_fraction'],
                p[f'vasculature_fraction-{x}']
            )
        for x in SHARED_PARAMS["blood"]
    }


def get_pvs_porosity(p):
    return {
        f"porosity-{x}":
            subnetwork_porosity(
                p['pvs_volume_fraction'],
                p[f'vasculature_fraction-{x.replace("pvs_", "")}']
            )
        for x in SHARED_PARAMS["blood"]
    }

In [50]:
get_vasculature_porosity(base_parameters)

{'porosity-arteries': 0.00658,
 'porosity-capillaries': 0.00329,
 'porosity-veins': 0.02303}

In [52]:
unpacked  = unpack_shared_parameters(base_parameters)
unpacked

{'brain_volume': 2450.0 <Unit('millimeter ** 3')>,
 'pressure_boundary-pvs_arteries': 4.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-ecs': 3.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-pvs_veins': 3.36 <Unit('millimeter_Hg')>,
 'pressure_boundary-arteries': 120.0 <Unit('millimeter_Hg')>,
 'pressure_boundary-veins': 7.0 <Unit('millimeter_Hg')>,
 'diffusion_free_inulin-ecs': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-pvs_arteries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-pvs_capillaries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-pvs_veins': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-arteries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-capillaries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-veins': 2.98 <Unit('millimeter ** 2 / second')>,
 'tortuosity-ecs': 1.7,
 'tortuosity-pvs_arteries': 1.7,
 'tortuosity-pvs_capillaries': 1.7,
 'tortuosity-pv

## Graveyard